In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在 Vertex AI 特征存储中配置 IAM 策略

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/vertex_ai_feature_store_iam_policy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fblob%2Fmain%2Fnotebooks%2Fofficial%2Ffeature_store%2Fvertex_ai_feature_store_iam_policy.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/tree/main/notebooks/official/feature_store/vertex_ai_feature_store_iam_policy.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Workbench 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/main/notebooks/official/feature_store/vertex_ai_feature_store_iam_policy.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何配置 IAM 策略来控制对 Vertex AI 特征存储中存储的资源和数据的访问。

了解有关 [Vertex AI 特征存储](https://cloud.google.com/vertex-ai/docs/featurestore/overview) 的更多信息。

执行的步骤包括：
- 创建一个 BigQuery 数据集，并设置一个`FeatureView`来运行周期性同步作业。
- 配置在线存储实例并设置在线服务。
- 配置 IAM 策略以管理访问控制。

## 开始使用

安装Vertex AI SDK和其他必需的包。

In [ ]:
! pip3 install --upgrade --user --quiet google-cloud-aiplatform google-cloud-bigquery

### 重新启动运行时（仅限 Colab）

要使用新安装的软件包，您必须在 Google Colab 上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。请等待直到完成再继续下一步。⚠️</b>
</div>

### 在 Google Colab 上验证您的笔记本环境

在 Google Colab 上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并初始化Vertex AI SDK

要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

在Vertex AI特征存储中配置IAM策略

In [ ]:
LOCATION_PARENT = "projects/" + PROJECT_ID + "/locations/" + LOCATION

配置必要的服务客户端：
- [FeaturestoreServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.featurestore_service.FeaturestoreServiceClient)
- [FeatureOnlineStoreAdminServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1beta1.services.feature_online_store_admin_service.FeatureOnlineStoreAdminServiceClient)
- [FeatureRegistryServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1beta1.services.feature_registry_service.FeatureRegistryServiceClient) 

配置必要的服务客户端：
- [FeaturestoreServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.featurestore_service.FeaturestoreServiceClient)
- [FeatureOnlineStoreAdminServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1beta1.services.feature_online_store_admin_service.FeatureOnlineStoreAdminServiceClient)
- [FeatureRegistryServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1beta1.services.feature_registry_service.FeatureRegistryServiceClient)

In [ ]:
from google.cloud.aiplatform_v1beta1 import (
    FeatureOnlineStoreAdminServiceClient, FeatureOnlineStoreServiceClient,
    FeatureRegistryServiceClient, FeaturestoreServiceClient)
from google.cloud.aiplatform_v1beta1.types import \
    feature_online_store_service as feature_online_store_service_pb2
from google.iam.v1 import iam_policy_pb2, policy_pb2

ENDPOINT = LOCATION + "-aiplatform.googleapis.com"
featurestore_service_client = FeaturestoreServiceClient(
    client_options={"api_endpoint": ENDPOINT}
)
feature_online_store_admin_service_client = FeatureOnlineStoreAdminServiceClient(
    client_options={"api_endpoint": ENDPOINT}
)
feature_registry_service_client = FeatureRegistryServiceClient(
    client_options={"api_endpoint": ENDPOINT}
)

生成此colab的示例数据。

In [ ]:
DATASET_ID = "test_data"+"_"+LOCATION.replace('-', '_')  # @param {type:"string"}
TABLE_ID = "tableA"  # @param {type:"string"}

!bq mk --dataset_id={DATASET_ID} --location={LOCATION}
!bq query --nouse_legacy_sql \
"CREATE TABLE {DATASET_ID}.{TABLE_ID} AS (" \
"SELECT * FROM UNNEST(ARRAY<STRUCT<entity_id STRING, feature_timestamp TIMESTAMP, feature1 INT64, feature2 INT64>>[" \
"('test', TIMESTAMP('2024-02-26 08:00:00 UTC'), 10, 20)," \
"('test', TIMESTAMP('2024-02-27 08:00:00 UTC'), 30, 40)," \
"('test', TIMESTAMP('2024-02-28 08:00:00 UTC'), 50, 60)]))"

创建 `FeatureOnlineStore` 和 `FeatureView` 资源

提供一个 `FeatureOnlineStore` 实例。

In [ ]:
FEATURE_ONLINE_STORE_ID = "test_feature_online_store"

feature_online_store_admin_service_client.create_feature_online_store(
    {
        "parent": LOCATION_PARENT,
        "feature_online_store_id": FEATURE_ONLINE_STORE_ID,
        "feature_online_store": {
            "bigtable": {
                "auto_scaling": {
                    "min_node_count": 1,
                    "max_node_count": 3,
                    "cpu_utilization_target": 50,
                }
            }
        },
    }
).result()

将BigQuery中的离线存储配置为定期数据同步到`FeatureOnlineStore`实例。

In [ ]:
FEATURE_VIEW_ID = "test_feature_view"
FEATURE_ONLINE_STORE_RESOURCE_ID = (
    LOCATION_PARENT + "/featureOnlineStores/" + FEATURE_ONLINE_STORE_ID
)
FEATURE_VIEW_RESOURCE_ID = (
    FEATURE_ONLINE_STORE_RESOURCE_ID + "/featureViews/" + FEATURE_VIEW_ID
)
create_feature_view_result = (
    feature_online_store_admin_service_client.create_feature_view(
        parent=FEATURE_ONLINE_STORE_RESOURCE_ID,
        feature_view_id=FEATURE_VIEW_ID,
        feature_view={
            "big_query_source": {
                "uri": f"bq://{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}",
                "entity_id_columns": ["entity_id"],
            },
            "sync_config": {"cron": "0 12 * * *"},
        },
    ).result()
)

## 配置IAM策略

为功能在线商店配置IAM策略。

In [ ]:
# A specific user email or a group email can be used to add members to the IAM Policy.
EXAMPLE_RESOURCE_VIEWER_EMAIL = (
    "google-cloud-eng-fte@google.com"  # @param {type:"string"}
)
EXAMPLE_DATA_VIEWER_EMAIL = "python-sample-owners@google.com"  # @param {type:"string"}
# A Service Account can also be added to the IAM Policy.
EXAMPLE_ADMIN_SERVICE_ACCOUNT = "samples@python-docs-samples-tests.iam.gserviceaccount.com"  # @param {type:"string"}
feature_online_store_admin_service_client.set_iam_policy(
    request=iam_policy_pb2.SetIamPolicyRequest(
        resource=FEATURE_ONLINE_STORE_RESOURCE_ID,
        policy=policy_pb2.Policy(
            bindings=[
                policy_pb2.Binding(
                    role="roles/aiplatform.featurestoreResourceViewer",
                    members=[f"group:{EXAMPLE_RESOURCE_VIEWER_EMAIL}"],
                ),
                policy_pb2.Binding(
                    role="roles/aiplatform.featurestoreDataViewer",
                    members=[f"group:{EXAMPLE_DATA_VIEWER_EMAIL}"],
                ),
                policy_pb2.Binding(
                    role="roles/aiplatform.admin",
                    members=[f"serviceAccount:{EXAMPLE_ADMIN_SERVICE_ACCOUNT}"],
                ),
            ]
        ),
    )
)

确认在线商店功能的更新IAM策略。

In [ ]:
feature_online_store_admin_service_client.get_iam_policy(
    request=iam_policy_pb2.GetIamPolicyRequest(
        resource=FEATURE_ONLINE_STORE_RESOURCE_ID
    )
)

为一个功能视图配置IAM策略。

In [ ]:
# A specific user email or a group email can be used to add members to the IAM Policy.
EXAMPLE_RESOURCE_VIEWER_EMAIL = (
    "google-cloud-eng-fte@google.com"  # @param {type:"string"}
)
EXAMPLE_DATA_VIEWER_EMAIL = "python-sample-owners@google.com"  # @param {type:"string"}
# A Service Account can also be added to the IAM Policy.
EXAMPLE_ADMIN_SERVICE_ACCOUNT = "samples@python-docs-samples-tests.iam.gserviceaccount.com"  # @param {type:"string"}
feature_online_store_admin_service_client.set_iam_policy(
    request=iam_policy_pb2.SetIamPolicyRequest(
        resource=FEATURE_VIEW_RESOURCE_ID,
        policy=policy_pb2.Policy(
            bindings=[
                policy_pb2.Binding(
                    role="roles/aiplatform.featurestoreResourceViewer",
                    members=[f"group:{EXAMPLE_RESOURCE_VIEWER_EMAIL}"],
                ),
                policy_pb2.Binding(
                    role="roles/aiplatform.featurestoreDataViewer",
                    members=[f"group:{EXAMPLE_DATA_VIEWER_EMAIL}"],
                ),
                policy_pb2.Binding(
                    role="roles/aiplatform.admin",
                    members=[f"serviceAccount:{EXAMPLE_ADMIN_SERVICE_ACCOUNT}"],
                ),
            ]
        ),
    )
)

请等一段时间，让新添加的IAM策略绑定生效。

* 注意：策略更改通常需要2分钟，因此这种延迟可能会缩短。有关更多详情，请查看IAM文档。

In [ ]:
# Wait 10 minutes for the newly updated IAM policy binding to become effective.
from time import sleep

sleep(600)

确认更新后的IAM策略。

In [ ]:
feature_online_store_admin_service_client.get_iam_policy(
    request=iam_policy_pb2.GetIamPolicyRequest(resource=FEATURE_VIEW_RESOURCE_ID)
)

根据需求运行批量同步。

In [ ]:
sync_response = feature_online_store_admin_service_client.sync_feature_view(
    feature_view=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}"
)

# Wait for sync completion
while True:
    feature_view_sync = feature_online_store_admin_service_client.get_feature_view_sync(
        name=sync_response.feature_view_sync
    )
    if feature_view_sync.run_time.end_time.seconds > 0:
        status = "Succeed" if feature_view_sync.final_status.code == 0 else "Failed"
        print(f"Sync {status} for {feature_view_sync.name}.")
        break
    else:
        print("Sync ongoing, waiting for 30 seconds.")
    sleep(30)

确认批量同步的状态。

In [ ]:
feature_online_store_admin_service_client.get_feature_view_sync(
    name=sync_response.feature_view_sync
)

开始在线服务

数据同步完成后，请使用`FetchFeatureValues` API来检索数据。

In [ ]:
data_client = FeatureOnlineStoreServiceClient(client_options={"api_endpoint": ENDPOINT})

从特色在线商店中读取同步的数据。

In [ ]:
data_client.fetch_feature_values(
    request=feature_online_store_service_pb2.FetchFeatureValuesRequest(
        feature_view=FEATURE_VIEW_RESOURCE_ID,
        data_key=feature_online_store_service_pb2.FeatureViewDataKey(key="test"),
    )
)

清理工作

要清理本项目中使用的所有谷歌云资源，您可以删除用于本教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的集群。

In [ ]:
# Delete feature view
feature_online_store_admin_service_client.delete_feature_view(
    name=FEATURE_VIEW_RESOURCE_ID
)

# Delete online store
feature_online_store_admin_service_client.delete_feature_online_store(
    name=FEATURE_ONLINE_STORE_RESOURCE_ID
)

# Delete test data
!bq rm -f {DATASET_ID}.{TABLE_ID}